<a href="https://colab.research.google.com/github/adudars/PedidON/blob/master/arquivos/9.9_ConsultasSelfJoin%26View.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9.9 CONSULTAS COM SELF JOIN E VIEW

## ESTABELECENDO CONEXÃO




### Instalando bibliotecas de acesso

In [1]:
#!pip install --user psycopg2-binary
#!pip install --user pandas

### Importando bibliotecas

In [3]:
import pandas as pd
import psycopg2

### Conectando com o banco através do psycopg2

In [97]:
conn = psycopg2.connect(host="lallah.db.elephantsql.com", database="nxthbnlu", user="nxthbnlu", password="buyAYKsY17pqexg1grIcJaWm0gHBsO2F")

## CONSULTA COM SELF JOIN

Uma vez que o SELF JOIN é utilizado para criar relacionamento entre informações de uma mesma tabela, visando portanto o relacionamento entre os registros pertencentes a mesma, e isso não ocorre em nosso banco de dados PedidON, substituímos, então, uma consulta com self join por uma criação de view, conforme orientado.

## VIEWS

### Funcionário destaque

In [66]:
cursor_view_destaque = conn.cursor()
cursor_view_destaque.execute("""create or replace view funcionario_destaque as
                        select nome_funcionario, count(Fk_id_Funcionario) as atendimentos
                        from comanda as c
                        inner join funcionario as f on 
                        (f.id_Funcionario = c.Fk_id_Funcionario)
                        where date_part('month',data) = date_part('month',(current_date + INTERVAL '-1 month')) and date_part('year',data) = date_part('year',(current_date + INTERVAL '-1 month'))
                        group by nome_funcionario
                        order by atendimentos desc
                        limit 1""")


In [58]:
destaque = pd.read_sql_query("""select * from funcionario_destaque""", conn)

In [59]:
destaque

,nome_funcionario,atendimentos
0,Matheus,2


### Frequência clientes no mês anterior

In [67]:
cursor_view_frequencia = conn.cursor()
cursor_view_frequencia.execute("""create or replace view frequencia_clientes as
                        select nome_cliente, count(distinct data) as frequencia
                        from cliente as c
                        inner join comanda as cd on
                        (c.id_cliente = cd.fk_id_cliente)
                        where date_part('month',data) = date_part('month',(current_date + INTERVAL '-1 month')) and date_part('year',data) = date_part('year',(current_date + INTERVAL '-1 month'))
                        group by nome_cliente
                        order by frequencia desc;""")


In [68]:
frequencia = pd.read_sql_query("""select * from frequencia_clientes""", conn)

In [69]:
frequencia

,nome_cliente,frequencia
0,Davi,1
1,Henrique,1
2,Isabella,1
3,Jorge,1


### Cardápio TOP 3 no mês anterior

In [70]:
cursor_view_top3 = conn.cursor()
cursor_view_top3.execute("""create or replace view cardapio_top_3 as
                            select ic.nome_item, ic.preco as valor, sum(quantidade) as total_pedidos
                            from pedido as p
                            inner join item_cardapio as ic on
                            (ic.id_itemcardapio = p.fk_id_itemcardapio)
                            inner join comanda as cd on
                            (p.fk_id_comanda = cd.id_comanda)
                            where date_part('month',data) = date_part('month',(current_date + INTERVAL '-1 month')) and date_part('year',data) = date_part('year',(current_date + INTERVAL '-1 month'))
                            group by ic.nome_item, ic.preco
                            order by total_pedidos desc
                            limit 3;""")


In [71]:
top3 = pd.read_sql_query("""select * from cardapio_top_3""", conn)

In [72]:
top3

,nome_item,valor,total_pedidos
0,Convento da Penha,115.0,4
1,Bons ventos te trazem,25.9,3
2,Barlavento,25.9,3


### Itens de cardápio

In [74]:
cursor_view_itens = conn.cursor()
cursor_view_itens.execute("""create or replace view Itens_cardapio as
                            select nome_item as nome, ic.descricao, preco, nome_categoria as categoria, t.descricao as tamanho
                            from item_cardapio as ic
                            inner join categoria as c on
                            (ic.fk_id_categoria = c.id_categoria)
                            inner join tamanho as t on
                            (ic.fk_id_tamanho = t.id_tamanho)""")


In [75]:
itens = pd.read_sql_query("""select * from itens_cardapio""", conn)

In [76]:
itens

,nome,descricao,preco,categoria,tamanho
0,Parque Moscoso,Polvo à galega - Polvo cozido servido com sal ...,145.00,principal,2 pessoas
1,Convento da Penha,Peixe do dia servido inteiro e assado com limã...,115.00,principal,2 pessoas
2,Ilha de Socó,Chocolate em mousse fria e semi congelada com ...,22.00,sobremesa,1 pessoa
3,Bons ventos te trazem,"Gin, redução de flor de sabugueiro, maracujá, ...",25.90,drink,1 pessoa
4,Barlavento,"Gin, xarope de morango, maracujá, suco de lara...",25.90,drink,1 pessoa
5,Cidade Sol,"Whiskey Jack Daniels, maracujá, mix de cítrico...",29.90,drink,1 pessoa
6,Reta da Penha,"Filé Mignon marinado na mostarda antiga, servi...",56.90,entrada,1 pessoa
7,Rua Sete,"Filé Mignon ao conhaque, servido com pães da casa",55.00,entrada,1 pessoa
8,Stella Artois,Stella Artois Long Neck,5.99,drink,long neck
9,Pepsi,Refrigerante,6.90,drink,lata


### Faturamento do dia corrente

In [106]:
cursor_view_faturamento = conn.cursor()
cursor_view_faturamento.execute("""create or replace view faturamento_diario as
                                select sum(preco * quantidade) as valor_total_dia, to_char(data, 'dd-mm-yyyy')
                                from item_cardapio as cp
                                inner join pedido as p on
                                (p.fk_id_itemcardapio = cp.id_itemcardapio)
                                inner join comanda as cd on
                                (p.fk_id_comanda = cd.id_comanda)
                                where data = current_date
                                group by data""")


In [107]:
faturamento = pd.read_sql_query("""select * from faturamento_diario""", conn)

In [108]:
faturamento

,valor_total_dia,to_char


**OBS:** De fato não há faturamento no dia corrente, por isso a consulta veio vazia

### Gasto cliente no mês anterior

In [103]:
cursor_view_gasto = conn.cursor()
cursor_view_gasto.execute("""create or replace view gasto_cliente as
                              select nome_cliente, sum(quantidade * preco) as valor_comanda
                              from comanda as cd
                              inner join pedido as p on
                              (cd.id_comanda = p.fk_id_comanda)
                              inner join item_cardapio as cp on
                              (p.fk_id_itemcardapio = cp.id_itemcardapio)
                              inner join cliente as c on
                              (cd.fk_id_cliente = c.id_cliente)
                              where date_part('month',data) = date_part('month',(current_date + INTERVAL '-1 month')) and date_part('year',data) = date_part('year',(current_date + INTERVAL '-1 month'))
                              group by nome_cliente
                              order by valor_comanda desc;""")


In [104]:
gasto = pd.read_sql_query("""select * from gasto_cliente""", conn)

In [105]:
gasto

,nome_cliente,valor_comanda
0,Henrique,964.90
1,Isabella,166.90
2,Davi,135.49
3,Jorge,6.90
